In [3]:
import cv2
import mediapipe as mp
import time
import numpy as np
import pycaw
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume


#############################
w_cam, h_cam = 640, 480
#############################

cap = cv2.VideoCapture(0)
cap.set(3, w_cam)
cap.set(4, h_cam)

mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)

#volume.GetMute()
#volume.GetMasterVolumeLevel()
vol_range = volume.GetVolumeRange()
print(vol_range)
min_vol = vol_range[0]
max_vol = vol_range[1]


pTime = 0.0
cTime = 0.0

while True:
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    #print(results.multi_hand_landmarks)
    if results.multi_hand_landmarks:
        for handlns in results.multi_hand_landmarks:
            list_lns = []
            for id, lm in enumerate(handlns.landmark):
                #print(id, lm)
                h, w, c = img.shape
                cx, cy = int(lm.x*w), int(lm.y*h)
                #print(id, cx, cy)
                if id==4 or id==8:
                    list_lns.append([cx,cy])
                    cv2.circle(img, (cx, cy), 10, 
                               (0, 255, 0), cv2.FILLED)
            x1,y1,x2,y2=list_lns[0][0], list_lns[0][1], list_lns[1][0], list_lns[1][1]
            Dist = np.sqrt((x1-x2)**2+(y1-y2)**2)
            cv2.line(img, (x1, y1),(x2,y2), 
                    (0, 255, 0),3, cv2.FILLED) 
            #print(Dist)
            if Dist<30:
                cv2.circle(img, ((x1+x2)//2, (y1+y2)//2), 15, 
                           (0, 0, 255), cv2.FILLED)
            # Hand Range 30-280
            vol = np.interp(Dist, [30,280], [min_vol, max_vol])
            #print(vol)
            volume.SetMasterVolumeLevel(vol, None)  

            
           
                
            
            mpDraw.draw_landmarks(img, handlns, mpHands.HAND_CONNECTIONS)

    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime

    cv2.putText(img, str(int(fps)), (10,70), cv2.FONT_HERSHEY_PLAIN,2,
                (0,255,0),0)

    cv2.imshow("Image", img)
    cv2.waitKey(1)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyWindow('Image')
        break




ImportError: cannot import name 'COMError' from 'ctypes' (/usr/lib/python3.10/ctypes/__init__.py)